In [ ]:
"""
Extract monthly air temperature data from the NetCDF file for the specified years and save each as a GeoTIFF raster with a date-based filename
"""

import os
import re
from datetime import datetime

# Define the NetCDF variable and dimension names
variable = "air"
x_dimension = "lon"
y_dimension = "lat"
band_dimension = ""
dimension = "time"
valueSelectionMethod = "BY_VALUE"

# Function to convert date strings from "dd.mm.yyyy" to "mm_yyyy"
def format_date(date_str):
    date_obj = datetime.strptime(date_str, "%d.%m.%Y")
    return date_obj.strftime("%m_%Y")

# Output directory and path to the input NetCDF file
outLoc = "temp_month"
inNetCDF = "air.mon.mean.v401.nc"

# Load NetCDF file properties
nc_FP = arcpy.NetCDFFileProperties(inNetCDF)
nc_Dim = nc_FP.getDimensions()  # Get all available dimensions in the NetCDF

# Iterate through all dimensions
for dimension in nc_Dim:
    if dimension == "time":  # Focus on the time dimension
        top = nc_FP.getDimensionSize(dimension)  # Number of time steps

        for i in range(0, top):
            dimension_values = nc_FP.getDimensionValue(dimension, i)  # Get the i-th date
            formatted_date = format_date(str(dimension_values))   # Format the date

            if int(formatted_date[-4]) == 2 and int(formatted_date[-2]) == 1:  # check for "201X"
                
                # Define the specific time step to extract
                dv1 = ["time", dimension_values]
                dimension_values = [dv1]

                layer_name  = formatted_date  # Name for the output raster layer

                # Create a raster layer from the NetCDF file
                arcpy.MakeNetCDFRasterLayer_md(
                    inNetCDF, variable, x_dimension, y_dimension, layer_name , 
                    band_dimension, dimension_values, valueSelectionMethod
                )

                # Define the output file name
                outname = os.path.join(outLoc, formatted_date + ".tif")
                
                # Save the raster to disk
                arcpy.CopyRaster_management(layer_name , outname)

                # Remove the raster layer from memory
                arcpy.Delete_management(layer_name )
                print(f"Processed: {outname}")

            else:
                print("Not 201X")  # Skip years outside of the 2010s


In [ ]:
"""
Calculate annual average raster files by averaging monthly raster data
"""

import arcpy
from arcpy.sa import *
import os

# Set environment settings
arcpy.env.workspace = "temp_month"  # Folder containing the raster files
arcpy.env.overwriteOutput = True

# Output folder same as input folder
output_folder = "temp_month"

# Iterate through the years
for year in range(2010, 2020):
    raster_list = []

    # Iterate through the months
    for month in range(1, 13):
        raster_name = f"{month:02d}_{year}.tif"  # File format: MM_YYYY.tif
        if arcpy.Exists(raster_name):
            raster_list.append(raster_name)

    # If all 12 monthly rasters exist, calculate the annual mean
    if len(raster_list) == 12:
        mean_raster = CellStatistics(raster_list, "MEAN", "DATA")
        output_raster = os.path.join(output_folder, f"annual_mean_{year}.tif")
        mean_raster.save(output_raster)
        print(f"Annual mean raster for {year} saved")
    else:
        print(f"Not all 12 monthly rasters are available for the year {year}!")


In [ ]:
"""
Focal statistics to fill NoData Cells, that appear especially at coastal regions
"""


import arcpy
from arcpy.sa import *

arcpy.CheckOutExtension("Spatial")

for year in range(2010, 2015):

    input_raster = fr"Z:\temp_month\annual_mean_{year}.tif"

    # Apply Focal Statistics using a 3x3 cell rectangular neighborhood
    focal_mean = FocalStatistics(input_raster, NbrRectangle(3, 3, "CELL"), "MEAN", "DATA")

    # Use the Con (conditional) tool to fill NoData cells:
    # If a cell is NoData, replace it with the focal mean; otherwise, keep the original value
    filled_raster = Con(IsNull(input_raster), focal_mean, input_raster)

    # Save the result to a new file
    output_path = fr"Z:\temp_month\annual_mean_filled_{year}.tif"
    filled_raster.save(output_path)
    
    print(f"Processed: {year}")

arcpy.CheckInExtension("Spatial")

In [ ]:
"""
Resample temperature data to match the resolution of population data
"""

import arcpy
import os

# Set the working directory
workspace = r"Z:\seminar_econometrics\temp_month"
arcpy.env.workspace = workspace

for year in range(2010, 2015):
    print(year)
    
    # Target cell size (matching the resolution of the population data)
    cell_size = 0.008333333333300002

    input_raster = f"annual_mean_filled_{year}.tif"

    out_raster = os.path.join(workspace, "r_" + input_raster)
    
    # Perform resampling using the Nearest Neighbor method (preserves original values)
    arcpy.Resample_management(input_raster, out_raster, cell_size, "NEAREST")

    print(f"Resampled: {input_raster} -> {out_raster}")



In [ ]:
"""
Calculate product raster of temperature and population
"""

import arcpy
from arcpy.sa import *

arcpy.CheckOutExtension("Spatial")

# Define input and output directories
input_folder = "temp_month"
output_folder = "product_temp"
arcpy.env.workspace = input_folder

population_raster = "glup90ag"

arcpy.env.overwriteOutput = True

# Loop through each year and calculate the product raster
for year in range(2010, 2015):
    print(year)
    
    raster = f"r_annual_mean_filled_{year}.tif"
    
    # Multiply temperature raster by the population raster (cell-by-cell)
    product_raster = Raster(raster) * Raster(population_raster)
    
    # Construct output filename
    output_name = "product_" + raster
    output_path = output_folder + "\\" + output_name

    product_raster.save(output_path)
    print(f"Created: {output_path}")

# Release the Spatial Analyst extension
arcpy.CheckInExtension("Spatial")


In [ ]:
"""
Zonal statistics for product raster
"""

import os
from arcpy import env
from arcpy.sa import *


arcpy.CheckOutExtension("Spatial")

arcpy.env.overwriteOutput = True

zone_data = "ntblnds.shp"

# Attribute field used to define zones (country code)
zone_field = "Iso3v10"

output_gdb = "temp_tables.gdb"


for year in range(2010, 2015):

    raster = f"product_r_annual_mean_filled_{year}.tif"

    out_table = os.path.join(output_gdb, f"ZonalStats_annual_mean_filled_{year}")

    # Perform Zonal Statistics as Table:
    # - "DATA" ensures only valid (non-NoData) cells are used
    # - "SUM" calculates the sum of raster values within each zone / country
    arcpy.gp.ZonalStatisticsAsTable_sa(zone_data, zone_field, raster, out_table, "DATA", "SUM")

    print(f"Processed: {raster}")


arcpy.CheckInExtension("Spatial")


In [ ]:
"""
Zonal statistics for population data (calculated only once)
"""

import os
from arcpy import env
from arcpy.sa import *

arcpy.CheckOutExtension("Spatial")

arcpy.env.overwriteOutput = True

raster = r"Z:\seminar_econometrics\population\glup90ag"

zone_data = r"Z:\seminar_econometrics\bnds\ntblnds.shp"

zone_field = "Iso3v10"

output_gdb = r"Z:\seminar_econometrics\temp_tables.gdb"

out_table = os.path.join(output_gdb, "ZonalStats_population_nw")


arcpy.gp.ZonalStatisticsAsTable_sa(zone_data, zone_field, raster, out_table, "DATA", "SUM")
    
print(f"Processed: {raster}")

arcpy.CheckInExtension("Spatial")

In [ ]:
"""
Join tables and calculate weighted temperature (temperature × population / population)
"""

import arcpy

gdb_path = "temp_tables.gdb"
arcpy.env.workspace = gdb_path

arcpy.env.overwriteOutput = True

# Name of the reference table (TableA) and the join field
tableA = "ZonalStats_population_nw"
join_field = "ZONE_CODE"

# list of all tables in the geodatabase
all_tables = arcpy.ListTables()

# Select all tables *except* TableA, and only those from 2010 to 2014
other_tables = [tbl for tbl in all_tables if tbl != tableA and tbl.endswith(("_2010", "_2011", "_2012", "_2013", "_2014"))]
#print(other_tables)

# Loop through each matching table, join with population table, and compute the weighted temperature
for tbl in other_tables:
    print("Processing table:", tbl)
    
    view_name = "view_" + tbl
    arcpy.MakeTableView_management(tbl, view_name)
    
    # Join the current table with the population table on the join field
    arcpy.AddJoin_management(view_name, join_field, tableA, join_field, "KEEP_COMMON")
    
    new_field = "w_temp"  # Weighted Annual Temperature

    arcpy.AddField_management(tbl, new_field, "DOUBLE")
    
    ratio_field_full = f"{tbl}.{new_field}"
    
    code_block = """def calc_ratio(val1, val2):
    if val2 != 0:
        return val1 / val2
    else:
        return 0"""

    expression = f"calc_ratio(!{tbl}.SUM!, !ZonalStats_population_nw.SUM!)"
    
    # Apply the calculation using the provided code block and expression
    arcpy.CalculateField_management(view_name, ratio_field_full, expression, "PYTHON3", code_block)
    
    arcpy.RemoveJoin_management(view_name, tableA)
    
    print("Processed table: ", tbl)

print("All tables have been processed.")